In [235]:
import requests
from bs4 import BeautifulSoup
import reverse_geocode
import folium
from geopy.geocoders import Nominatim
import numpy as np
import matplotlib as mpl
import pandas as pd
from sklearn import datasets
import geocoder

In [271]:
Raleigh_Neighborhoods=[]
response_obj=requests.get('https://en.wikipedia.org/wiki/Category:Neighborhoods_in_Raleigh,_North_Carolina').text
soup=BeautifulSoup(response_obj,'lxml')
mydivs = soup.findAll("div", {"class": "mw-category-group"})
for div in mydivs:
    for l in div.findAll('a'):
        for title in l:
            Raleigh_Neighborhoods.append(title)
Raleigh_Neighborhoods.remove('Raleigh, North Carolina neighborhoods')
Raleigh_Neighborhoods=[s.replace('Historic District','') for s in Raleigh_Neighborhoods]
Raleigh_Neighborhoods=[s.replace('(North Carolina)','') for s in Raleigh_Neighborhoods]
Raleigh_Neighborhoods=[s.replace('(Raleigh, North Carolina)','') for s in Raleigh_Neighborhoods]
Raleigh_Neighborhoods=[s.strip(' ') for s in Raleigh_Neighborhoods]
Raleigh_Neighborhoods=[s+(',Raleigh, North Carolina') for s in Raleigh_Neighborhoods]
Raleigh_Neighborhoods

['Battery Heights,Raleigh, North Carolina',
 'Bloomsbury,Raleigh, North Carolina',
 'Boylan Heights,Raleigh, North Carolina',
 'Cameron Park,Raleigh, North Carolina',
 'Cameron Village,Raleigh, North Carolina',
 'Cameron Village,Raleigh, North Carolina',
 'Capitol Area,Raleigh, North Carolina',
 'Capitol Heights,Raleigh, North Carolina',
 'Centennial Campus of North Carolina State University,Raleigh, North Carolina',
 'Depot,Raleigh, North Carolina',
 'East Raleigh–South Park,Raleigh, North Carolina',
 'Fayetteville Street (Raleigh),Raleigh, North Carolina',
 'Fayetteville Street,Raleigh, North Carolina',
 'Five Points Historic Neighborhoods,Raleigh, North Carolina',
 'Glenwood Avenue,Raleigh, North Carolina',
 'Glenwood–Brooklyn,Raleigh, North Carolina',
 'Hayes Barton,Raleigh, North Carolina',
 'Hi-Mount,Raleigh, North Carolina',
 'Historic Oakwood,Raleigh, North Carolina',
 'Longview Gardens,Raleigh, North Carolina',
 'Madonna Acres,Raleigh, North Carolina',
 'Maiden Lane,Raleigh, N

In [273]:
df_ral=pd.DataFrame(Raleigh_Neighborhoods)
df_ral=df_ral.rename(columns={0:'Neighborhoods'})
df_ral=df_ral.drop_duplicates(keep='first')
df_ral

,Neighborhoods
0,"Battery Heights,Raleigh, North Carolina"
1,"Bloomsbury,Raleigh, North Carolina"
2,"Boylan Heights,Raleigh, North Carolina"
3,"Cameron Park,Raleigh, North Carolina"
4,"Cameron Village,Raleigh, North Carolina"
6,"Capitol Area,Raleigh, North Carolina"
7,"Capitol Heights,Raleigh, North Carolina"
8,Centennial Campus of North Carolina State Univ...
9,"Depot,Raleigh, North Carolina"
10,"East Raleigh–South Park,Raleigh, North Carolina"


In [319]:
geolocator=Nominatim(user_agent='Raleigh')
city=[]
lat=[]
long=[]
from geopy.exc import GeocoderTimedOut
try:
    for i in Raleigh_Neighborhoods:
        location = geolocator.geocode(i)
        if location is not None:
            city.append(i)
            lat.append(location.latitude)
            long.append(location.longitude)
except GeocoderTimedOut as e:
    print("Error: geocode failed on input %s with message %s"%(location, e.msg))

In [322]:
df=pd.DataFrame({'Neighborhoods':city,'latitude':lat,'longitude':long})

In [326]:
df=df.drop_duplicates(subset='latitude',keep='first')

In [327]:
df

,Neighborhoods,latitude,longitude
0,"Battery Heights,Raleigh, North Carolina",35.777058,-78.617563
1,"Bloomsbury,Raleigh, North Carolina",35.808897,-78.648599
2,"Boylan Heights,Raleigh, North Carolina",35.774159,-78.652102
3,"Cameron Park,Raleigh, North Carolina",35.785779,-78.655470
4,"Cameron Village,Raleigh, North Carolina",35.790361,-78.660413
6,"Capitol Area,Raleigh, North Carolina",35.789884,-78.621851
8,"Depot,Raleigh, North Carolina",35.780398,-78.639099
9,"Fayetteville Street (Raleigh),Raleigh, North C...",35.768875,-78.641336
11,"Glenwood Avenue,Raleigh, North Carolina",35.886724,-78.744499
12,"Hayes Barton,Raleigh, North Carolina",35.802755,-78.646583
